Importação de todos os pacotes que serão utilizados ao longo do modelo.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [2]:
#leitura dos dados
df_review = pd.read_csv("IMDB Dataset.csv")
#checagem do balanceamento
df_review.value_counts(subset='sentiment')

sentiment
negative    25000
positive    25000
Name: count, dtype: int64

Dataset já veio perfeitamente balanceado. Nesse caso, não há necessidade de tomar qualquer ação nesse sentido.  
De forma semelhante, os dados já estão limpos, não exigindo esse passo tampouco.

In [ ]:
#separação dos dados em treino e teste, utilizando proporção 80/20
treino, teste = train_test_split(df_review, test_size=0.2, random_state=42)
treino_x, treino_y = treino['review'], treino['sentiment']
teste_x, teste_y = teste['review'], teste['sentiment']

treino_x: variáveis independentes (review/análises) utilizadas para treinar o modelo. 
treino_y: variáveis dependentes/labels (sentiment) que o modelo deve prever.  
teste_x: variáveis independentes que serão utilizadas para teste de precisão do modelo.  
teste_y: labels/etiquetas usadas para testar a precisão da previsão do modelo contra as categorias de fato

Para transformar a representação textual em vetores numéricos, utilizaremos a técnica de bag of words (BOW).  
A escolha se dá por: 1) a frequência das palavras nas análises importa, podendo ser indicador de seu sentimento; 2) a ordem em si das palavras possui menor relevância.  
A BOW será representada por Term Frequency, Inverse Document Frequency (TF-IDF), forma de ponderar o peso de cada palavra de um documento (no caso, análise individual) levando em conta sua presença no corpus total (o conjunto de documentos/análises).

In [ ]:
tfidf = TfidfVectorizer(stop_words='english') #remoção de stop words/palavras vazias da língua inglesa
treino_x_vetor = tfidf.fit_transform(treino_x) #encontra parâmetros internos do modelo e os aplica, vetorizando o treino
teste_x_vetor = tfidf.transform(teste_x) #apenas vetoriza o teste para uso
treino_x_vetor

Teste de diferentes modelos de aprendizagem supervisionada (classificação) para selecionar aquele com maior precisão. Testaremos Support Vector Machines (SVM), árvore de decisões, Naive Bayes e regressão logística, comparando suas precisões médias (mean accuracy), F1 score, classification report e matriz de confusão.

In [ ]:
#inserção dos dados nos algoritmos
#SVM
svc = SVC(kernel='linear')
svc.fit(treino_x_vetor, treino_y)
#Decision Tree
dec_tree = DecisionTreeClassifier()
dec_tree.fit(treino_x_vetor, treino_y)
#Naive Bayes
gnb = GaussianNB()
gnb.fit(treino_x_vetor.toarray(), treino_y)
#Regressão logística
log_reg = LogisticRegression()
log_reg.fit(treino_x_vetor, treino_y)

In [ ]:
#Mean Accuracy
svc.score(teste_x_vetor, teste_y)
dec_tree.score(teste_x_vetor, teste_y)
gnb.score(teste_x_vetor.toarray(), teste_y)
log_reg.score(teste_x_vetor, teste_y)

Como o resultado do Naive Bayes e das árvores de decisão foram consideravelmente abaixo daqueles apresentados pelo SVM e regressão logística, utilizaremos apenas esses dois últimos para os demais testes de comparação.

In [ ]:
#F1 score
f1_score(teste_y, svc.predict(teste_x_vetor), 
         labels=['positive', 'negative'], average=None)
f1_score(teste_y, log_reg.predict(teste_x_vetor), 
         labels=['positive', 'negative'], average=None)

In [ ]:
#Classification report
print(classification_report(teste_y,
                            svc.predict(teste_x_vetor),
                            labels=['positive', 'negative']))
print(classification_report(teste_y,
                            log_reg.predict(teste_x_vetor),
                            labels=['positive', 'negative']))

In [ ]:
#Confusion matrix
conf_mat_svc = confusion_matrix(teste_y,
                            svc.predict(teste_x_vetor),
                            labels=['positive', 'negative'])
conf_mat_logreg = confusion_matrix(teste_y,
                            log_reg.predict(teste_x_vetor),
                            labels=['positive', 'negative'])